# Tensorflow

[TF](https://tensorflow.google.cn/api_docs/python/)使用的是计算图的方式,所以代码看起来会比较难以理解.并且TF是不需要写参数更新的,它将根据你输入的损失函数自行进行参数更新.



(1)

我们需要先定义占位符[tf.placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder)用于反应张量:

```python
tf.placeholder(tf.float32,shape=[None,n])``` 

shape中一般只要放入特征值的大小,TF会自动将输入的X塞进去.

(2) 创建初始化变量使用[tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable)

```python 
tf.Variable(tf.random_normal(shape=[1,n],seed=1))```

tf.random_normal:从正态分布输出随机值,seed为随机数种子,shape用于初始化变量定义形状.

(3) 矩阵相乘使用[tf.matmul](https://www.tensorflow.org/api_docs/python/tf/linalg/matmul)

```python
tf.matmul```


(4) 计算损失函数可以使用[tf.nn.sigmoid_cross_entropy_with_logits(logits=Z,labels=y)](https://www.tensorflow.org/api_docs/python/tf/nn/sigmoid_cross_entropy_with_logits)

```python

tf.nn.sigmoid_cross_entropy_with_logits(logits=Z,labels=y)```

注意这里logits项会直接帮你计算sigmoid,所以传入的并不是sigmoid作用之后的值,而是线性值.


(5) 梯度下降优化器使用[tf.train.GradientDescentOptimizer](https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer)

```python
tf.train.GradientDescentOptimizer```

minimize(cost):使用优化器后目的就是最小化minimize,也就是相当于开始参数更新.

(6) tf.equal:计算两者是否相等,再使用[tf.cast](https://www.tensorflow.org/api_docs/python/tf/dtypes/cast)转换类型,最后求平均.

将需要的代码定义完毕了之后,就可以开始初始化所有的变量[tf.global_variables_initializer()](https://www.tensorflow.org/api_docs/python/tf/initializers/global_variables)

```python
tf.global_variables_initializer()```


(7) 当初始化完毕了之后就可以开始运行计算图了[tf.Session()](https://www.tensorflow.org/api_docs/python/tf/Session)

```python
sess.run(init)
...
tf.Session()```

一般这一步都是定死的

(8) 计算图运行....

(9) 运行完毕之后就可以计算正确率:

```python
accuracy.eval```

将需要的数据喂入,即可计算正确率.





### 所以TF的整体结构就是:

(1) 定义变量和占位符

(2) 编写需要的代码(构建网络架构)

(3) 初始化所有代码

(4) 运行计算图

(5) 得到参数更新结果

**Ps:**

[Tensor类型为什么可以自动求导](https://www.zhihu.com/question/54554389)

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import h5py

In [2]:
def load_data():
    '''
    create train set and test set
    make sure you have .h5 file in your dataset
    
    Returns:
    -------
        train_set_x_orig: original train set shape is (209, 64, 64, 3) 
        train_set_y_orig: original train label shape is (209,)
        test_set_x_orig: original test set shape is (50, 64, 64, 3)
        test_set_y_orig: original test label shape is (50,)
        classes: cat or non-cat.
        
    Note:
    ----
        (209, 64, 64, 3): 209 picture,64 width,64 height,3 channel.
    '''
    train_dataset = h5py.File('../data_set/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('../data_set/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [3]:
train_x_orig, train_y_orig, test_x_orig, test_y_orig, classes = load_data()

In [4]:
train_x_tensor = train_x_orig.reshape(train_x_orig.shape[0],-1) / 255 
test_x_tensor = test_x_orig.reshape(test_x_orig.shape[0],-1) / 255
train_y_tensor = train_y_orig.reshape(1,-1)
test_y_tensor = test_y_orig.reshape(1,-1)
print('Train_x\'s shape:{}'.format(train_x_tensor.shape))
print('Test_x\'s shape:{}'.format(test_x_tensor.shape))
print("Train_y's shape:{}".format(train_y_tensor.shape))
print("Test_y's shape:{}".format(test_y_tensor.shape))

Train_x's shape:(209, 12288)
Test_x's shape:(50, 12288)
Train_y's shape:(1, 209)
Test_y's shape:(1, 50)


In [5]:
def initial_parameters(n):
    """
    initialization parameters
    
    Parameters:
    ----------
        n: fetures
    Returns:
        W:TF variable,weigths
        b:TF variable,bais
    
    
    """
    W = tf.Variable(tf.random_normal(shape=[1,n],seed=1))
    b = tf.Variable(tf.zeros(shape=[1,1]))
    
    return W,b

In [8]:
def LR_Tensorflow(data,Labels,test_data,test_label,alpha,Iter):
    """
    Build LR Tensorflow
    
    Parameters:
    ----------
        data: training set
        Labels: training set
        test_data: test set
        test_label: test labels
        alpha: learning rate
        Iter: iterate
    
    """
   
    m,n = data.shape
    
    # create placeholder.
    X = tf.placeholder(tf.float32,shape=[None,n]) # shape is (None,n)
    y = tf.placeholder(tf.float32,shape=[1,None])
    
    # initialization parameters
    W,b = initial_parameters(n)
    
    # calculate learn Regression.
    Z = tf.matmul(W,tf.transpose(X)) + b
    
    # compute cost 
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Z,labels=y))
    
    # using optimization of Gradient Descent and minimize cost.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=alpha).minimize(cost)
    
    # calculate correct rate
    correct_ = tf.equal(tf.round(tf.sigmoid(Z)),y)
        
    accuracy = tf.reduce_mean(tf.cast(correct_,tf.float32))
    
    # initialization variable.
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        
        sess.run(init)

        for iter_ in range(Iter):
            
            # return value equal list size.
            # need feed dict,how many placeholder how many value.
            _,c = sess.run([optimizer,cost],feed_dict={X:data,y:Labels})
           
            if iter_ % 100 == 0:
                
                print('after iter {},loss {}'.format(iter_,c))
        
        # input data and labels to calculateaccuracy.
        correct_rate_test = accuracy.eval({X:test_data,y:test_label })
        correct_rate_train = accuracy.eval({X:data,y:Labels})
        print("The test set correct is ",correct_rate_test)
        print("The train set correct is ",correct_rate_train)

In [9]:
LR_Tensorflow(train_x_tensor,train_y_tensor,test_x_tensor,test_y_tensor,0.1,1100)

after iter 0,loss 7.985492706298828
after iter 100,loss 3.4039981365203857
after iter 200,loss 14.464579582214355
after iter 300,loss 1.3415228128433228
after iter 400,loss 9.29128646850586
after iter 500,loss 4.443009376525879
after iter 600,loss 1.3032474517822266
after iter 700,loss 0.31220778822898865
after iter 800,loss 0.1305931806564331
after iter 900,loss 0.22720423340797424
after iter 1000,loss 0.043595489114522934
The test set correct is  0.6
The train set correct is  0.9952153


# Summary

可以看出TF集成了许多可用的函数,整体的逻辑思路也是一样的,只是我们要先建立一些tensor,比如placeholder.接着还是一样的先正向传播再计算loss,定义优化器optimizer,之后反向传播,接着更新参数.只要熟悉TF框架的定义,那么模型代码就会变得简单.

另外由于tensor的存在,使得我们不需要进行backward,这是非常方便的,因为在之后随着模型深度加深以及一些其他优化措施下,推导backward是非常困难的,而且如果只是使用Numpy定义,计算速度也是非常的慢.